# PCA Visual

In [45]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [46]:


# Add paths to system path
sys.path.extend(str(path) for path in TOOL_PATHS.values())

# import write_lc as lc
# import bea_validation as bea
# import azi_lc_name_lm as write_lcname
# import coordinate_mapper as cm
# import meta_visual as mv
import plot_3d as pl

In [47]:
def fit_plane_pca(points):
    """
    Fits a plane to a set of 3D points using PCA

    Args:
        points (numpy.ndarray): An array of shape (n, 3) representing the 3D points.

    Returns:
        tuple: A tuple containing the normal vector (numpy.ndarray) and distance from origin (float) of the fitted plane.

    Source: https://stats.stackexchange.com/questions/163356/fitting-a-plane-to-a-set-of-points-in-3d-using-pca
  """
    # Center the data
    centroid_data = np.mean(points, axis=0)
    centered_points = points - centroid_data
    # print(centroid_data)
    # print(centered_points)
    # Covariance matrix
    cov_matrix = np.cov(centered_points.T)
    # print(cov_matrix)
    # Solve for eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    # print(eigenvalues)
    # print(eigenvectors)
    # Normal vector is the eigenvector corresponding to the smallest eigenvalue
    normal_vector = eigenvectors[:, np.argmin(eigenvalues)]

    # Distance from origin (assuming the first coordinate is x)
    # distance_from_origin = np.abs(np.dot(points[0], normal_vector)) / np.linalg.norm(normal_vector)
    d = -np.dot(centroid_data, normal_vector)/np.linalg.norm(normal_vector)
    return normal_vector, d, centroid_data, eigenvalues, eigenvectors

In [7]:
import numpy as np
from sklearn.decomposition import PCA

def generate_points_on_plane(normal, point_on_plane, num_points=100, noise=0.0):
    """
    Generate random points on a plane defined by a normal vector and a point on the plane.

    Parameters:
        normal (array-like): A 3D vector normal to the plane.
        point_on_plane (array-like): A point on the plane (x, y, z).
        num_points (int): Number of points to generate.
        noise (float): Magnitude of random noise to add to the points.

    Returns:
        np.ndarray: Array of shape (num_points, 3) with points on the plane.
    """
    normal = np.array(normal)
    point_on_plane = np.array(point_on_plane)
    normal = normal / np.linalg.norm(normal)  # Normalize the normal vector

    # Create two orthogonal vectors in the plane
    v1 = np.random.randn(3)
    v1 -= v1.dot(normal) * normal  # Make it orthogonal to the normal
    v1 /= np.linalg.norm(v1)

    v2 = np.cross(normal, v1)  # Second orthogonal vector

    # Generate random points in the plane
    coefficients = np.random.rand(num_points, 2) - 0.5  # Random values for linear combinations
    points = point_on_plane + coefficients[:, 0, None] * v1 + coefficients[:, 1, None] * v2

    # Add noise
    points += np.random.normal(scale=noise, size=points.shape)

    return points

def fit_plane(points):
    """
    Fit a plane to a set of 3D points using PCA and return the plane's normal vector.

    Parameters:
        points (np.ndarray): Array of shape (n, 3) representing the 3D points.

    Returns:
        tuple: A tuple containing:
            - normal (np.ndarray): Normal vector to the best-fit plane.
            - point_on_plane (np.ndarray): Centroid of the points, lying on the plane.
    """
    centroid = np.mean(points, axis=0)
    pca = PCA(n_components=3)
    pca.fit(points - centroid)
    normal = pca.components_[-1]  # Normal vector is the last principal component
    return normal, centroid

# Example Usage
if __name__ == "__main__":
    # Define a plane
    normal = [1, 2, -1]
    point_on_plane = [0, 0, 0]

    # Generate points on the plane
    points = generate_points_on_plane(normal, point_on_plane, num_points=5, noise=0.01)

    # Fit a plane to the points
    fitted_normal, centroid = fit_plane(points)

    # Results
    print("Original Normal Vector:", normal)
    print("Fitted Normal Vector:", fitted_normal)
    print("Centroid of the Plane:", centroid)


Original Normal Vector: [1, 2, -1]
Fitted Normal Vector: [ 0.41410333  0.81687728 -0.40153448]
Centroid of the Plane: [-0.03601829 -0.03124174 -0.10719179]


In [68]:
normal = [1, 6, 7]
point_on_plane = [1, 1, 7]
points = generate_points_on_plane(normal, point_on_plane, num_points=101, noise=0.01)

normal_vector, d, centr, eigenvalues, eigenvectors = fit_plane_pca(points)
print(f"!normal:{normal_vector}")
print(f"!Distance: {d}")
print(f"!Centro: {centr}")

!normal:[0.1050449  0.64470906 0.7571762 ]
!Distance: -6.0502080442126625
!Centro: [1.00184609 1.01341456 6.98861335]


In [69]:
eigenvalues, eigenvectors = eigen_plane_pca(points)
print(f"{eigenvalues}, \n\n{eigenvectors}")

[1.03589973e-01 5.60235856e-02 8.29860812e-05], 

[[ 0.66458771  0.73978966  0.1050449 ]
 [ 0.52089071 -0.55948466  0.64470906]
 [-0.53572011  0.37374881  0.7571762 ]]


In [70]:
vec1 = eigenvectors[:, 0]
vec2 = eigenvectors[:, 1]
vec3 = eigenvectors[:, 2]

In [71]:
fig1 = pl.plot_3d_point(points)
# fig1.show()

In [72]:
# list1=[centr,normal_vector+centr]
list1=[centr, vec1+centr]
list2=[centr, vec2+centr]   
list3=[centr, vec3+centr] 
# list4=[[0,0,0],[1,1,1]]
fig2=pl.plot_lines_from_points(list1,list2,list3)
# fig2.show()

In [75]:
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 30},title = '3D Vector Plot',width=700,height=400,)
fig3

In [67]:
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 30},title = '3D Vector Plot',width=700,height=400,)
fig3

In [11]:
# list=[[1,2,3],[4,6,6],[7,8,9],[10,11,12],[2,5,6]]
# fig2=pl.plot_3d_line(list)
# fig2.show()